# introduction to the vectorizing principle

## part 5a: generating SQL and dataframe transformation code with one syntax

While pandas is the most popular library for describing data transformations in python, there are more which you will also see in following parts. Relational databases are also a means of highly efficient processing of tabular data. They are typically instructed with a declarative language called SQL which allows for highly sophisticated query engines to first plan how to execute the query. From a large search space of possible execution plans they chose the supposedly best one by using computation cost estimators.

For having a quick look at the data or for searching bugs in previously executed transformation code, it is highly efficient to write ad-hoc SQL scripts by hand. However, for better software engineering practices such as testing, programmatically creating SQL is the preferred option. In fact, it would be ideal to have one syntax from which both dataframe transformation code and SQL can be generated. This is what we show in this part.

The library [pydiverse.transform](https://github.com/pydiverse/pydiverse.transform/) is used here even though it is early stage and still lacks essential features. It is intended as a proof of concept for this idea.

Pandas uses the fluent interface technique chaining functional methods with multiple function calls: `df.assign(x=lambda df: df.int_col*2).loc[lambda df: df.x == 4]`

pydiverse.transform uses the pipe syntax instead, similar to its precursors [dplyr from R](https://dplyr.tidyverse.org/) and [siuba](https://siuba.org/): `df >> mutate(x=λ.int_col*2) >> filter(λ.x == 4)`

pydiverse.transform also supports creating custom verbs with the `@verb` decorator which then can be used within a pipe just like builtin verbs such as `mutate` or `filter`.

Detail: In SQL, joining a table does not add any new columns to the select statement. It rather brings a new table within the namespace that can be used for creating select columns. pydiverse.transform supports this feature by dropping columns with an empty `select()` statement which does not remove them from the namespace for further use in `mutate()` statements.

#### let's get started:

In [20]:
import pandas as pd
import numpy as np
import sqlalchemy as sa

from pydiverse.transform import λ, Table, verb
from pydiverse.transform.lazy.sql_table import SQLTableImpl
from pydiverse.transform.eager.pandas_table import PandasTableImpl

from pydiverse.transform.core.alignment import aligned, eval_aligned
from pydiverse.transform.core.verbs import alias, arrange, collect, filter, group_by, join, mutate, select, summarise, show_query, ungroup, left_join

In [21]:
# Load data...
titanic_dataset = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv')

engine = sa.create_engine('sqlite:///:memory:')
titanic_dataset.to_sql('titanic', engine, index = False, if_exists = 'replace')

# Create table objects
titanic_p = Table(PandasTableImpl('titanic', titanic_dataset))
titanic_s = Table(SQLTableImpl(engine, 'titanic'))

In [22]:
# Choose a backend ...
# titanic_p -> Pandas
# titanic_s -> SQLite
titanic = titanic_p

# ... and preview it
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.25,S,Third,man,True,<NA>,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.925,S,Third,woman,False,<NA>,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.05,S,Third,man,True,<NA>,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0,S,Second,man,True,<NA>,Southampton,no,True
887,1,1,female,19.0,0,0,30.0,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,<NA>,1,2,23.45,S,Third,woman,False,<NA>,Southampton,no,False
889,1,1,male,26.0,0,0,30.0,C,First,man,True,C,Cherbourg,yes,True


In [23]:
# Let's look at the average fare people payed that survived vs those that didn't survive
who_survived_fare = (titanic
    >> group_by(λ.who, λ.survived)
    >> summarise(avg_fare = λ.fare.mean())
)

who_survived_fare

,who,survived,avg_fare
0,child,0,32.633703
1,child,1,32.891329
2,man,0,21.490736
3,man,1,42.076422
4,woman,0,20.967174
5,woman,1,54.813801


In [24]:
# We now can analyze how much more each category of person would have had to pay to survive.
# For that we split our table into two seperate taples representing the two different survival outcomes.
dead  = who_survived_fare >> filter(λ.survived == 0) >> alias('dead')
alive = who_survived_fare >> filter(λ.survived == 1) >> alias('alive')

print(dead)
print()
print(alive)

Table: dead, backend: PandasTableImpl
     who  survived   avg_fare
0  child         0  32.633703
2    man         0  21.490736
4  woman         0  20.967174

Table: alive, backend: PandasTableImpl
     who  survived   avg_fare
1  child         1  32.891329
3    man         1  42.076422
5  woman         1  54.813801


In [25]:
# This is the SQL style of this ...  (with an explicit join)

price_to_survive_lazy = (dead
 >> left_join(alive, dead.who == alive.who, validate='1:?')
 >> select(λ.who)
 >> mutate(price_to_survive = alive.avg_fare - dead.avg_fare))

price_to_survive_lazy

,who,price_to_survive
0,child,0.257626
2,man,20.585686
4,woman,33.846627


In [26]:
# ... but we can also do it in a more dataframe style (assuming aligned row ordering).

# Because we know that the 'dead' and the 'alive' table are aligned with each
# other (they have the same order and same number of rows), we can perform
# row wise operations on their columns without explicitly joining the two tables.

# The downside of this approach is that it only works in some limited
# cases with the SQL backend.

price_to_survive = eval_aligned(
    alive.avg_fare - dead.avg_fare
)

# We can then add this new column back to our dataframe.
dead >> select(λ.who) >> mutate(price_to_survive = price_to_survive)

,who,price_to_survive
0,child,0.257626
2,man,20.585686
4,woman,33.846627


In [27]:
# Alternatively we can also look at how likely a person was to survive based on their age.
survival_by_age = (titanic
 >> mutate(age_bucket = round(λ.age + 4.999, -1))
 >> group_by(λ.age_bucket)
 >> summarise(samples=λ.age_bucket.count(), survival_likelyhood=λ.survived.mean())
 >> arrange(λ.age_bucket))

# As you can see, children ages 0 to 10 were most and people between 70 and 80 were 
# least likely to survive.
survival_by_age

,age_bucket,samples,survival_likelyhood
0,10.0,64,0.59375
1,20.0,115,0.382609
2,30.0,230,0.365217
3,40.0,155,0.445161
4,50.0,86,0.383721
5,60.0,42,0.404762
6,70.0,17,0.235294
7,80.0,5,0.2
8,<NA>,0,0.293785


In [28]:
# So far we've been using the pandas as the backend for our calculations.
# You can now go back to the third cell of this notebook and replace
# `titanic = titanic_p` with `titanic = titanic_s` to switch to the 
# SQLite backand and then re-run all the cells.

# You will see that everything (with the exception of the eager style
# aligned evaluation) still works and that it produces exactly the same
# output.

# Additionally, we now can display the appropriate SQL query that correspond
# to some of our calculations.

In [29]:
price_to_survive_lazy >> show_query()

No query to show for PandasTableImpl


,who,price_to_survive
0,child,0.257626
2,man,20.585686
4,woman,33.846627


In [30]:
survival_by_age >> show_query()

No query to show for PandasTableImpl


,age_bucket,samples,survival_likelyhood
0,10.0,64,0.59375
1,20.0,115,0.382609
2,30.0,230,0.365217
3,40.0,155,0.445161
4,50.0,86,0.383721
5,60.0,42,0.404762
6,70.0,17,0.235294
7,80.0,5,0.2
8,<NA>,0,0.293785


In [31]:
# It's also super easy to define custom verbs.
# By adding the @verb decorator to a functions it can be used inside a pipe
# just like any of the builtin verbs.

# In the following example we want to take a table, trim all leading and
# trailing whitespace from all string columns and then concatenate them
# along the column axis.

@verb
def trim_all_str(tbl):
    for col in tbl:
        if col._.dtype == 'str':
            tbl[col] = col.strip()
    return tbl

@verb
def join_all_str(tbl, seperator=''):
    return tbl >> summarise(**{
        col._.name: col.join(seperator)
        for col in tbl if col._.dtype == 'str'
    })


# Create table and transform using our custom verbs.

str_df = pd.DataFrame({
    'col1': ['  a', 'b  ', ' c '],
    'col2': ['qua', ' nt', 'co '],
    
    'some_other_col': [1, 2, 3],
})

str_tbl = Table(PandasTableImpl('str_tbl', str_df))
trimmed_tbl = str_tbl >> trim_all_str()
joined_tbl = trimmed_tbl >> join_all_str()

print(f"""
Original Table
--------------
{str_tbl}

Trimmed Table
-------------
{trimmed_tbl}

Joined Table
------------
{joined_tbl}
""")


Original Table
--------------
Table: str_tbl, backend: PandasTableImpl
  col1 col2  some_other_col
0    a  qua               1
1  b     nt               2
2   c   co                3

Trimmed Table
-------------
Table: str_tbl, backend: PandasTableImpl
  col1 col2  some_other_col
0    a  qua               1
1    b   nt               2
2    c   co               3

Joined Table
------------
Table: str_tbl, backend: PandasTableImpl
  col1     col2
0  abc  quantco


Next: [vectorization05b.ipynb](vectorization05b.ipynb): generating SQL and dataframe transformation code with one syntax - an example pipeline